### Biblioteki

In [ ]:
using Images
using PlotlyJS
using Colors, ColorVectorSpace, FixedPointNumbers, Statistics
using Interact
using Distances, Plots
using BenchmarkTools

# Przygotowanie Danych

### Wczytanie pliku Image0026-rgb.tif

In [ ]:
# wczytanie pliku TIFF przy użyciu biblioteki Images.jl
img26 = load("dataset/rgb/FISH_Brachy/Image0026-rgb.tif");

In [ ]:
# wnętrze piksela
dump(img26[rand(1:end)])

In [ ]:
# wyświetlenie serii obrazów
mosaicview(img26; fillvalue=0.5, npad=2, ncol=6, rowmajor=true)

### Wczytanie plików Experiment-392

In [ ]:
# wczytanie ścieżek do plików
file_red = "dataset/rgb/Centr_tel Brachy/Experiment-392.czi - C=r.tif"
file_green = "dataset/rgb/Centr_tel Brachy/Experiment-392.czi - C=g.tif"
file_blue = "dataset/rgb/Centr_tel Brachy/Experiment-392.czi - C=b.tif"

# wczytanie plików TIFF przy użyciu biblioteki Images
tif_red=load(file_red);
tif_green=load(file_green);
tif_blue=load(file_blue);

### Funkcja generująca współrzędne dla układu 3D (3D Array)

In [ ]:
function get_coords(img)
    y_size, x_size, z_size = size(img)

    z_list = zeros(0)

    for i in 1:z_size
        append!(z_list, fill(i ,y_size * x_size))
    end

    x_list = zeros(0)
    for i in 1:x_size
        append!(x_list, fill(i ,y_size))
    end  

    append!(x_list, repeat(x_list,z_size-1))

    y_list = collect(1:y_size)
    append!(y_list, repeat(y_list,z_size * x_size-1));
    
    return hcat(y_list,x_list,z_list)
end

### Połączenie współrzędnych, kolorów i rozmiarów pikseli

In [ ]:
# wydobycie współrzędnych
coords_matrix = get_coords(img26);

# spłaszczenie danych do postaci Wektora (Vector)
img26_flat = vec(img26)

# przekształcenie typu kolorów z N0f8 na Float64
img26_flat = RGB{Float64}.(img26_flat)

# utworzenie kolumny rozmiarów pikseli (same jedynki)
ones_array = ones(length(img26_flat))

# połączenie danych
img26_arr=hcat(coords_matrix, red.(img26_flat), green.(img26_flat), 
    blue.(img26_flat), ones_array);

Połączenie współrzędnych z kolorami i rozmiarem dla exp392

In [ ]:
coords_matrix = get_coords(tif_red)

# spłaszczenie układu 3d na układ 1d
tif_red_flat = vec(tif_red)
tif_green_flat = vec(tif_green)
tif_blue_flat = vec(tif_blue)

# utworzenie kolumny zapisującej rozmiar
ones_array = ones(length(tif_red_flat))

# połączenie wektorów w układ 2d
tif_arr=hcat(coords_matrix, Float64.(tif_red_flat*4), Float64.(tif_green_flat*4), Float64.(tif_blue_flat*4), ones_array);

### Usuwanie czarnego tła

In [ ]:
function clear_black_bg(tif_arr, threshold)
    tif_arr = tif_arr[vec(mapslices(col -> (col[4] .> threshold)
                | (col[5] .> threshold)
                | (col[6] .> threshold), 
            tif_arr, dims = 2)), :]
    return tif_arr
end

# odfiltrowanie pliku img26
println("Rozmiar danych przed usunięciem tła: "
    ,size(img26_arr))

img26_arr_cl=clear_black_bg(img26_arr,0.315);

println("Rozmiar danych przed usunięciem tła: "
    ,size(img26_arr_cl))

In [ ]:
tif_arr_cl=clear_black_bg(tif_arr,0.392156862);

### funkcje pomocnicze

### Obliczanie centroidu - Julia

In [ ]:
function get_centroid(array_2d)
    length = size(array_2d, 1)
    sum_y = sum(array_2d[:,1])
    sum_x = sum(array_2d[:,2])
    sum_z = sum(array_2d[:,3])
    
    return hcat(sum_y/length, sum_x/length, sum_z/length, 
        mean(array_2d[:,4]), mean(array_2d[:,5]), 
        mean(array_2d[:,6]), length)
end

### Obliczanie medoidu - Julia

In [ ]:
function get_medoid(n)
    
    # tworzenie macierzy wzajemnych odległości
    distMatrix=pairwise(Euclidean(),n[:,1:6],dims=1)
    
    # wybieranie piksela o najmniejszej sumie odległości
    medoid=n[argmin(sum(distMatrix,dims=2))[1],:]
    
    # dodanie rozmiaru grupy
    medoid=reshape(medoid, 1, length(medoid))
    medoid[7]=size(n,1)
    
    return medoid
end

### Obliczanie mediany - Julia

In [ ]:
function get_median(arr)
    # zapisanie rozmiaru zbioru
    length = size(arr, 1)
    
    # zwracanie median kolumn zbioru
    return hcat(median(arr[:,1]), median(arr[:,2]), 
        median(arr[:,3]), median(arr[:,4]), 
        median(arr[:,5]), median(arr[:,6]), length)
end

### Funkcje znajdujące sąsiadów - Julia

znadowanie sąsiadów po promieniu i różnicy w kolorze

In [ ]:
function get_neighbours(tif_arr, pixel, radius, zradius, color_diff)
    
    n = tif_arr[vec(mapslices(col -> 
                (col[1] .<= pixel[1]+radius) & (col[1] .>= pixel[1]-radius)
                & (col[2] .<= pixel[2]+radius) & (col[2] .>= pixel[2]-radius)
                & (col[3] .<= pixel[3]+zradius) & (col[3] .>= pixel[3]-zradius)
                & (col[4] .<= pixel[4]+color_diff) & (col[4] .>= pixel[4]-color_diff)
                & (col[5] .<= pixel[5]+color_diff) & (col[5] .>= pixel[5]-color_diff)
                & (col[6] .<= pixel[6]+color_diff) & (col[6] .>= pixel[6]-color_diff), 
                tif_arr, dims = 2)), :]
    return n
end

znajdowanie sąsiadów po promieniu

In [ ]:
function get_neighbours2(tif_arr, pixel, radius, zradius)
    
    n = tif_arr[vec(mapslices(col -> (col[1] .<= pixel[1]+radius) & (col[1] .>= pixel[1]-radius)
                                    & (col[2] .<= pixel[2]+radius) & (col[2] .>= pixel[2]-radius)
                                    & (col[3] .<= pixel[3]+zradius) & (col[3] .>= pixel[3]-zradius), 
                                    tif_arr, dims = 2)), :]
    return n
end

znajdowanie sąsiadów po różnicy w kolorze

In [ ]:
function get_neighbours_by_color(tif_arr, pixel, color_diff)
    
    n = tif_arr[vec(mapslices(col -> (col[4] .<= pixel[4]+color_diff) & (col[4] .>= pixel[4]-color_diff)
                                    & (col[5] .<= pixel[5]+color_diff) & (col[5] .>= pixel[5]-color_diff)
                                    & (col[6] .<= pixel[6]+color_diff) & (col[6] .>= pixel[6]-color_diff), 
                                    tif_arr, dims = 2)), :]
    return n
end

### Usuwanie podzbioru - Julia

In [ ]:
function remove_neighbours(tif_arr, n)
    tif_arr=tif_arr[vec(mapslices(row -> !(row in eachrow(n)), 
                tif_arr, dims = 2)),:]
    return tif_arr
end

### Metoda Centroidów - wariant I - Julia

In [ ]:
function centroid_method(tif_arr,radius,zradius,color_diff)
    
    # utworzenie pustego zbioru centrodów
    centroids = Array{Float64}(undef, 0, 7)

    # zapętlanie procesu, aż zbiór danych będzie pusty
    while size(tif_arr, 1)>0

        # losowanie startowego piksela
        pixel = tif_arr[rand(1:end), :] 
        
        # utworzenie macierzy pikseli sąsiadujących
        n=get_neighbours(tif_arr, pixel, radius, zradius, color_diff)
        
        # wyznaczanie centroidu ze zbioru sąsiadującego
        centroid = get_centroid(n)

        # dodanie centroidu do zbioru centroidów
        centroids = [centroids; centroid]

        # usuwanie zbioru sąsiadów ze zbioru danych
        tif_arr=remove_neighbours(tif_arr,n)
    end
    println(size(centroids))
    return centroids
end

### Centroid Method II - różnica koloru - z usuwaniem

In [ ]:
function centroid_method2(tif_arr,radius,zradius,color_diff)
    
    # utworzenie pustego zbioru centrodów
    centroids = Array{Float64}(undef, 0, 7)

    # zapętlanie procesu, aż zbiór danych będzie pusty
    while size(tif_arr, 1)>0

        # losowanie startowego piksela
        pixel = tif_arr[rand(1:end), :] 
        
        # utworzenie macierzy pikseli sąsiadujących na podstawie promienia
        n=get_neighbours2(tif_arr, pixel, radius, zradius)
        
        # usuwanie zbioru sąsiadującego ze zbioru danych
        tif_arr=remove_neighbours(tif_arr,n)
        
        # wyodrębnenie pikseli spełniających dozwoloną różnicę kolorów
        n=get_neighbours_by_color(n, pixel,color_diff)
        
        # wyznaczanie centroidu ze zbioru sąsiadującego
        centroid = get_centroid(n)

        # dodawanie centroidu do zbioru centroidów
        centroids = [centroids; centroid]

    end
    println(size(centroids))
    return centroids
end

### Centroid Method III - bez różnicy koloru + nie ma outlierów

In [ ]:
function centroid_method3(tif_arr,radius,zradius)
    
    # utworzenie pustego zbioru centrodów
    centroids = Array{Float64}(undef, 0, 7)

    # zapętlanie procesu, aż zbiór danych będzie pusty
    while size(tif_arr, 1)>0

        # losowanie startowego piksela
        pixel = tif_arr[rand(1:end), :] 
        
        # utworzenie macierzy pikseli sąsiadujących
        n=get_neighbours2(tif_arr, pixel, radius, zradius)
        
        # usuwanie zbioru sąsiadującego ze zbioru danych
        tif_arr=remove_neighbours(tif_arr,n)
        
        # wyznaczanie reprezentanta i jego zapis
        centroid = get_centroid(n)
        centroids = [centroids; centroid]

    end
    println(size(centroids))
    return centroids
end

### Medoid Method I - różnica koloru - bez usuwania

In [ ]:
function medoid_method(tif_arr,radius,zradius,color_diff)
    
    # utworzenie pustego zbioru medoidów
    medoids = Array{Float64}(undef, 0, 7)

    # zapętlanie procesu, aż zbiór danych będzie pusty
    while size(tif_arr, 1)>0

        # losowanie startowego piksela
        pixel = tif_arr[rand(1:end), :] 

        # utworzenie macierzy pikseli sąsiadujących
        n=get_neighbours(tif_arr, pixel, radius, zradius, color_diff)
        
        # wyznaczanie reprezentanta i jego zapis
        medoid=get_medoid(n)
        medoids=[medoids;medoid]
        
        # usuwanie sąsiadów z całego zbioru
        tif_arr=remove_neighbours(tif_arr,n)

    end
    println(size(medoids))
    return medoids
end

### Medoid Method II - różnica koloru - z usuwaniem

In [ ]:
function medoid_method2(tif_arr,radius,zradius,color_diff)
# defining empty matrix where centroids will be stored
    medoids = Array{Float64}(undef, 0, 7)

    # zapętlanie procesu, aż zbiór danych będzie pusty
    while size(tif_arr, 1)>0

        # losowanie startowego piksela
        pixel = tif_arr[rand(1:end), :] 

        # utworzenie macierzy pikseli sąsiadujących
        n=get_neighbours2(tif_arr, pixel, radius, zradius)
        
        # usuwanie zbioru sąsiadującego ze zbioru danych
        tif_arr=remove_neighbours(tif_arr,n)
        
        # wyodrębnenie pikseli spełniających dozwoloną różnicę kolorów
        n=get_neighbours_by_color(n, pixel, color_diff)
        
        # wyznaczanie reprezentanta i jego zapis
        medoid=get_medoid(n)
        medoids=[medoids;medoid]
        
    end
    println(size(medoids))
    return medoids
end

### Medoid Method III - bez różnicy koloru + nie ma outlierów

In [ ]:
function medoid_method3(tif_arr,radius,zradius)
# defining empty matrix where centroids will be stored
    medoids = Array{Float64}(undef, 0, 7)

    # zapętlanie procesu, aż zbiór danych będzie pusty
    while size(tif_arr, 1)>0

        # losowanie startowego piksela
        pixel = tif_arr[rand(1:end), :] 

        # utworzenie macierzy pikseli sąsiadujących
        n=get_neighbours2(tif_arr, pixel, radius, zradius)
        
        # usuwanie zbioru sąsiadującego ze zbioru danych
        tif_arr=remove_neighbours(tif_arr,n)
        
        # wyznaczanie reprezentanta i jego zapis
        medoid=get_medoid(n)
        medoids=[medoids;medoid]
        
    end
    println(size(medoids))
    return medoids
end

### Median Method I - różnica koloru - bez usuwania

In [ ]:
function median_method(tif_arr,radius,zradius,color_diff)
# defining empty matrix where centroids will be stored
    medians = Array{Float64}(undef, 0, 7)

    # zapętlanie procesu, aż zbiór danych będzie pusty
    while size(tif_arr, 1)>0

        # losowanie startowego piksela
        pixel = tif_arr[rand(1:end), :] 

        # utworzenie macierzy pikseli sąsiadujących
        n=get_neighbours(tif_arr, pixel, radius, zradius, color_diff)

        # wyznaczanie reprezentanta i jego zapis
        median=get_median(n)
        medians=[medians;median]
        
        # usuwanie zbioru sąsiadującego ze zbioru danych
        tif_arr=remove_neighbours(tif_arr,n)

    end
    println(size(medians))
    return medians
end

### Median Method II - różnica koloru - z usuwaniem

In [ ]:
function median_method2(tif_arr,radius,zradius,color_diff)
# defining empty matrix where centroids will be stored
    medians = Array{Float64}(undef, 0, 7)

    # zapętlanie procesu, aż zbiór danych będzie pusty
    while size(tif_arr, 1)>0

        # losowanie startowego piksela
        pixel = tif_arr[rand(1:end), :] 

        # utworzenie macierzy pikseli sąsiadujących
        n=get_neighbours2(tif_arr, pixel, radius, zradius)
        
        # usuwanie zbioru sąsiadującego ze zbioru danych
        tif_arr=remove_neighbours(tif_arr,n)
        
        # wyodrębnenie pikseli spełniających dozwoloną różnicę kolorów
        n=get_neighbours_by_color(n, pixel, color_diff)
        
        # wyznaczanie reprezentanta i jego zapis
        median=get_median(n)
        medians=[medians;median]
        
    end
    println(size(medians))
    return medians
end

### Median Method III - bez różnicy koloru + nie ma outlierów

In [ ]:
function median_method3(tif_arr,radius,zradius)
# defining empty matrix where centroids will be stored
    medians = Array{Float64}(undef, 0, 7)

    # zapętlanie procesu, aż zbiór danych będzie pusty
    while size(tif_arr, 1)>0

        # losowanie startowego piksela
        pixel = tif_arr[rand(1:end), :] 

        # utworzenie macierzy pikseli sąsiadujących
        n=get_neighbours2(tif_arr, pixel, radius, zradius)
        
        # usuwanie zbioru sąsiadującego ze zbioru danych
        tif_arr=remove_neighbours(tif_arr,n)
        
        # wyznaczanie reprezentanta i jego zapis
        median=get_median(n)
        medians=[medians;median]
        
    end
    println(size(medians))
    return medians
end

In [ ]:
function plotly_vis(matrix, smin, smax)
    # przygotowanie kolorów do wczytania do wykresu
    color_arr= RGB{Float64}.(matrix[:,4],matrix[:,5],matrix[:,6])
    return PlotlyJS.plot(PlotlyJS.scatter3d(
            ;x=matrix[:,2], 
            y=matrix[:,1], 
            z=matrix[:,3],
            text=matrix[:,7],
            customdata=color_arr,
            type="scatter3d",
            mode="markers",
            marker=attr(
                sizemode="area",
                size=matrix[:,7],
                color=color_arr,
                sizeref=(maximum(matrix[:,7])) / (smax ^ 2),
                sizemin=smin,
                opacity=1,
                line=attr(width=0)
            ),

            hovertemplate="<br>x:%{x}<br>y:%{y}<br>z:%{z}"
                *"<br>color: %{customdata}<br>size: %{text}"
        )
    )
end

### Widżet Julia

In [ ]:
function tif_reduction_widget(tif_arr)
    bg_threshold = slider(0.1:0.005:.6, label = "bg_threshold")
    radius = slider(1:15, label = "zradius")
    zradius = slider(1:5, label = "zradius")
    color_diff = slider(0:0.01:.3, label = "color_diff")
    smin = slider(0:20, label = "smin")
    smax = slider(40:60, label = "smax")
    method = dropdown(OrderedDict(
            "centroid method" => "centroid_method", 
            "centroid method 2" => "centroid_method2", 
            "centroid method 3" => "centroid_method3",
            "medoid method" => "medoid_method", 
            "medoid method 2" => "medoid_method2", 
            "medoid method 3" => "medoid_method3", 
            "median method" => "median_method", 
            "median method 2" => "median_method2", 
            "median method 3" => "median_method3")
            ,label="method")
    update = button("compute")
    
    output= Observable{Any}((undef, 0, 0))
    plt=Plots.plot()
    Interact.@map (&update; tif_arr=clear_black_bg(tif_arr, bg_threshold[]);
        if  method[] == "centroid_method"
            output[] = centroid_method(tif_arr,radius[],zradius[],color_diff[])
        elseif method[] == "centroid_method2"
            output[] = centroid_method2(tif_arr,radius[],zradius[],color_diff[])
        elseif method[] == "centroid_method3"
            output[] = centroid_method3(tif_arr,radius[],zradius[])
        elseif method[] == "medoid_method"
            output[] = medoid_method(tif_arr,radius[],zradius[],color_diff[])
        elseif method[] == "medoid_method2"
            output[] = medoid_method2(tif_arr,radius[],zradius[],color_diff[])
        elseif method[] == "medoid_method3"
            output[] = medoid_method3(tif_arr,radius[],zradius[])
        elseif method[] == "median_method"
            output[] = median_method(tif_arr,radius[],zradius[],color_diff[])
        elseif method[] == "median_method2"
            output[] = median_method2(tif_arr,radius[],zradius[],color_diff[])
        elseif method[] == "median_method3"
            output[] = median_method3(tif_arr,radius[],zradius[])end)
    plt = Interact.@map plotly_vis(&output,&smin,&smax)
    wdg = Widget(["bg_threshold" => bg_threshold,"radius" => radius,
                  "zradius" => zradius, "color_diff" => color_diff, 
                  "update" => update, "smin" => smin, "smax" => smax, 
                  "method"=>method], output = output);
    @layout! wdg hbox(plt, vbox(:bg_threshold,:radius, :zradius,
            :color_diff, pad(1em,:method), :update, :smin, :smax))
end

In [ ]:
tif_reduction_widget(img26_arr)

### Pomiary z podrozdziału Metody redukcji danych

test metod w wariancie II

In [ ]:
test=centroid_method(img26_arr_cl,5,2,0.0393);

In [ ]:
@time centroid_method2(img26_arr_cl,1,1,0.0393);

In [ ]:
@time medoid_method2(img26_arr_cl,1,1,0.0393);

In [ ]:
@time median_method2(img26_arr_cl,1,1,0.0393);

test metod w wariancie III

In [ ]:
@time centroid_method3(img26_arr_cl,1,1);

In [ ]:
@time medoid_method3(img26_arr_cl,1,1);

In [ ]:
@time median_method3(img26_arr_cl,1,1);

inne testy

In [ ]:
@benchmark centroid_method2($img26_arr_cl,1,1,0.0393)

In [ ]:
@btime centroid_method2(img26_arr_cl,1,1,0.0393)

In [ ]:
@benchmark centroid_method($img26_arr,5,1,0.0393)

In [ ]:
@benchmark medoid_method($img26_arr,10,1,0.0393)

In [ ]:
@benchmark get_centroid($img26_arr)

### Analiza porównówcza Testy

In [ ]:
centroid_method(img26_arr_cl,5,1,0.0393)

In [ ]:
@time centroid_method(img26_arr_cl,5,1,0.0393)
@time centroid_method(img26_arr_cl,4,1,0.0393)
@time centroid_method(img26_arr_cl,3,1,0.0393)

In [ ]:
@time centroid_method(img26_arr_cl,5,1,01965)
@time centroid_method(img26_arr_cl,4,1,01965)
@time centroid_method(img26_arr_cl,3,1,01965)

In [ ]:
@time medoid_method(img26_arr_cl,5,1,0.0393)
@time medoid_method(img26_arr_cl,4,1,0.0393)
@time medoid_method(img26_arr_cl,3,1,0.0393)

In [ ]:
@time medoid_method(img26_arr_cl,5,1,01965)
@time medoid_method(img26_arr_cl,4,1,01965)
@time medoid_method(img26_arr_cl,3,1,01965)

In [ ]:
@time median_method(img26_arr_cl,5,1,0.0393)
@time median_method(img26_arr_cl,4,1,0.0393)
@time median_method(img26_arr_cl,3,1,0.0393)

In [ ]:
@time median_method(img26_arr_cl,5,1,01965)
@time median_method(img26_arr_cl,4,1,01965)
@time median_method(img26_arr_cl,3,1,01965)

II

In [ ]:
@time centroid_method2(img26_arr_cl,3,1,0.0393)
@time centroid_method2(img26_arr_cl,2,1,0.0393)
@time centroid_method2(img26_arr_cl,1,1,0.0393)

In [ ]:
@time centroid_method2(img26_arr_cl,3,1,0.00393)
@time centroid_method2(img26_arr_cl,2,1,0.00393)
@time centroid_method2(img26_arr_cl,1,1,0.00393)

In [ ]:
@time medoid_method2(img26_arr_cl,3,1,0.0393)
@time medoid_method2(img26_arr_cl,2,1,0.0393)
@time medoid_method2(img26_arr_cl,1,1,0.0393)
@time medoid_method2(img26_arr_cl,3,1,0.00393)
@time medoid_method2(img26_arr_cl,2,1,0.00393)
@time medoid_method2(img26_arr_cl,1,1,0.00393)

In [ ]:
@time median_method2(img26_arr_cl,3,1,0.0393)
@time median_method2(img26_arr_cl,2,1,0.0393)
@time median_method2(img26_arr_cl,1,1,0.0393)
@time median_method2(img26_arr_cl,3,1,0.00393)
@time median_method2(img26_arr_cl,2,1,0.00393)
@time median_method2(img26_arr_cl,1,1,0.00393)

III

In [ ]:
@time centroid_method3(img26_arr_cl,3,1,0.0393)
@time centroid_method3(img26_arr_cl,2,1,0.0393)
@time centroid_method3(img26_arr_cl,1,1,0.0393)
@time centroid_method3(img26_arr_cl,3,1,0.00393)
@time centroid_method3(img26_arr_cl,2,1,0.00393)
@time centroid_method3(img26_arr_cl,1,1,0.00393)

In [ ]:
@time medoid_method3(img26_arr_cl,3,1,0.0393)
@time medoid_method3(img26_arr_cl,2,1,0.0393)
@time medoid_method3(img26_arr_cl,1,1,0.0393)
@time medoid_method3(img26_arr_cl,3,1,0.00393)
@time medoid_method3(img26_arr_cl,2,1,0.00393)
@time medoid_method3(img26_arr_cl,1,1,0.00393)

In [ ]:
@timeit median_method3(img26_arr_cl,3,1,0.0393)
@timeit median_method3(img26_arr_cl,2,1,0.0393)
@timeit median_method3(img26_arr_cl,1,1,0.0393)
@timeit median_method3(img26_arr_cl,3,1,0.00393)
@timeit median_method3(img26_arr_cl,2,1,0.00393)
@timeit median_method3(img26_arr_cl,1,1,0.00393)